# **<div align="center">CLEAN BINANCE COIN DATA</div>**

In [ ]:
import numpy as np
import pandas as pd

In [2]:
def fill_missing_prices(df: pd.DataFrame):

    '''
    Fill NA in price_usd if there is a price_use and change values the next day

    Parameters:
    df (pd.DataFrame): A DataFrame

    Returns:
    pd.DataFrame: A dataframe without the colums
    '''

    # Check that date column is a datetime and it is sortes
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').reset_index(drop=True)
    
    # Iterates over rows with null values in price_usd
    for i in range(len(df) - 1):
        if pd.isna(df.loc[i, 'price_usd']):
            next_day_price = df.loc[i + 1, 'price_usd']
            next_day_change = df.loc[i + 1, 'change']
            # Check if the calculation is possible
            if not pd.isna(next_day_price) and not pd.isna(next_day_change):
                # Calculation: price_t-1 = price_t / (1 + change_t / 100)
                df.loc[i, 'price_usd'] = next_day_price / (1 + next_day_change / 100)

    return df


In [3]:
def full_rest_variables(df: pd.DataFrame):

    '''
    Impute NAs using values of previous days.

    Parameters:
    df (pd.DataFrame): A DataFrame

    Returns:
    pd.DataFrame: A dataframe without the columns
    '''
    df = df.sort_values("date").reset_index(drop=True)

    # Impute values
    features_to_fill = df.columns.difference(["date", "price_usd", "change"])
    df[features_to_fill] = df[features_to_fill].ffill()

    #Drop change variable and the rest of data with NA to avoid impute with fabricated data the target
    df = df.drop(columns=["change"])
    df = df.dropna(subset=["price_usd"]).reset_index(drop=True)

    return df

In [ ]:
def export_dataset(df: pd.DataFrame, path: str, include_index: bool = False) -> None:

    '''
    Export the DataFrame to a CSV file.

    Parameters:
    df (pd.DataFrame): The DataFrame to export.
    path (str): The file path where to save the CSV.
    include_index (bool): Whether to include the index in the CSV file (default False).

    Returns:
    None: This function prints status messages but does not return a value.
    '''
    
    if not isinstance(path, str):
        raise ValueError("The path must be a string.")

    try:
        df.to_csv(path, index=include_index)
        print(f"Data exported successfully to {path}")
    except Exception as e:
        print(f"Error exporting data: {e}")

In [19]:
def time_plots(df: pd.DataFrame, features: list = [None]) -> None:

    '''
    Display subplot of temporal variables.

    Parameter:
        - df: pd.DataFrame
        - features: a list. By default: None

    Returns:
        - None
    '''
    fig, ax = plt.subplots(len(features), 1, figsize = (10, 6), sharex = True) #sharex = True is for sahering 
    # x axis

    for i, var in enumerate(features):
        ax[i].plot(train['date'], train[var], label = var)
        ax[i].legend(loc = "upper right")
        ax[i].set_ylabel(var)

    # for the last subplot
    ax[-1].set_xlabel("Date")

    plt.tight_layout() # Avoid overlapping
    plt.show()

## **Import dataset**

In [5]:
binance = pd.read_csv("../data/raw/binancecoin.csv")
binance['date'] = pd.to_datetime(binance['date'])
binance.head()

,date,price_usd,market_cap,volume,change
0,2025-02-05,571.03,8.327533e+10,1.241403e+09,-2.180040
1,2025-02-06,580.05,8.457818e+10,4.711889e+08,1.491920
2,2025-02-07,586.11,8.556300e+10,1.075487e+09,1.422947
3,2025-02-08,NaN,NaN,NaN,NaN
4,2025-02-09,NaN,NaN,NaN,NaN


## **Data description**

In [6]:
binance.shape

(136, 5)

In [7]:
binance.columns

Index(['date', 'price_usd', 'market_cap', 'volume', 'change'], dtype='object')

In [8]:
binance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        136 non-null    datetime64[ns]
 1   price_usd   92 non-null     float64       
 2   market_cap  92 non-null     float64       
 3   volume      92 non-null     float64       
 4   change      92 non-null     float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 5.4 KB


## **EDA for all dataset**

In [9]:
binance.duplicated().sum()

np.int64(0)

In [10]:
binance.isna().sum()

date           0
price_usd     44
market_cap    44
volume        44
change        44
dtype: int64

These NAs is due to days that I don't collect data. We can impute these data depending on the day. If we have a NA but the next day we have a data, we can impute the value of y using change. The rest of data must be imputed using a metrics or imputer.

In [11]:
binance_filled = fill_missing_prices(binance)

In [12]:
binance_filled = full_rest_variables(binance_filled)

In [13]:
binance_filled.isna().sum()

date          0
price_usd     0
market_cap    0
volume        0
dtype: int64

## **Export dataframe**

In [22]:
export_dataset(binance_filled, "../data/processed/binance_cleared.csv", include_index = False)

Data exported successfully to ../data/processed/binance_cleared.csv
